# This Notebook contains complete code for different statistical approach for classifying the data with its test accuracy.

---
The code is written in python and divided in many section


# Section 1: 
Reading data from the mnist binary data file.

In [0]:
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels


Importing required python Module and load data

In [0]:
import numpy as np
import pandas as pd
import numpy
import math,copy
import csv
import collections
import matplotlib.pyplot as plt


# Section 2: 
Bayesian Classifier with Maximum Likelihood Estimation

# PART A
Fashion Minst dataset

In [0]:
mus = []
training_freq = {}
training_data = []
test_data = []
sigs = []
dets = 1
priori = []
m0 = 1
s0 = 1
n = 1

In [0]:
#
def cal_priori():
    global training_freq
    global training_data
    global test_data
    global m0,s0
    global sigs
    global dets
    global priori
    global mus
    dic = training_freq.copy()
    for i in dic.keys():
        dic[i] = float(dic.get(i))/float(len(training_data))
    priori = dic.copy()
    return dic

In [0]:
#
def mean_mat():
    global training_freq
    global training_data
    global test_data
    global m0,s0
    global sigs
    global dets
    global priori
    global mus
    global n
    dic = {}
    for y,x in training_data:
        if y not in dic:
            dic[y] = []
        dic[y].append(x)
    for j in dic.keys():
        dic[j] = np.mean(dic.get(j),axis=0)
    ns0 = np.array(n*s0)
    sigs = sigs.astype(float)
    ns0 = ns0.astype(float)
    for va in dic.keys():
        mn0 = np.matmul(np.matmul(n*s0,np.linalg.inv(np.array(ns0+sigs))),dic.get(va))
        mn1 = np.matmul(np.matmul(sigs,np.linalg.inv(np.array(ns0+sigs))),m0)
        mn = np.array(mn0+mn1)
        dic[va] = mn
    mus = dic
    return dic

In [0]:
def mulvar_nor(X,y):
    global training_freq
    global training_data
    global test_data
    global m0,s0
    global sigs
    global dets
    global priori
    global mus
    #print dets
    #c = math.log(math.pow(2*math.pi,-0.5*len(X)))+math.log(math.pow(np.real(dets),-0.5))
    #math.log(2*math.pi) = 1.8378770664093453
    c = -0.5*len(X)*1.8378770664093453+ -0.5*abs(dets)
    xut = np.array([X-mus.get(y)])
    sig_i = np.linalg.inv(sigs)
    pd = np.matmul(xut , sig_i)
    xu = np.array([X-mus.get(y)]).T
    e = (-0.5)*np.linalg.det(np.matmul(pd,xu))
    return c+e


In [0]:
def mul_exp(X,y):
    return lam.get(y)*math.exp(-1*np.matmul(lam.get(y),X))


In [0]:
def train_rn():
    global training_freq
    global training_data
    global test_data
    global m0,s0
    global sigs
    global dets
    global priori
    global mus
    X_train, y_train = load_mnist('drive/ml1/data/fashion', kind='train')
    #X_train = np.array(pca(X_train))
    #X_train = np.array(X_train).real.astype(float)
    sigs = np.cov(X_train,rowvar=0)
    dets = np.linalg.slogdet(sigs)[-1]
    X_test, y_test = load_mnist('drive/ml1/data/fashion', kind='t10k')
    #X_test = pca(X_test)
    #X_test = np.array(X_test).real.astype(float)
    training_data = [x for x in zip(y_train,X_train[:-len(X_train)/10])]
    val_data = [x for x in zip(y_train,X_train[-len(X_train)/10:])]
    m0 = np.mean(X_train[-len(X_train)/10:],axis=0)
    s0 = np.cov(X_train[-len(X_train)/10:],rowvar=0)
    n=len(X_train)-len(X_train)/10
    sn = np.matmul(np.matmul(sigs,np.linalg.inv(np.array(n*s0+sigs))),s0)
    sigs = np.array(sn+sigs)
    test_data = [x for x in zip(y_test,X_test)]
    unique, counts = np.unique(y_train, return_counts=True)
    training_freq = dict(zip(unique,counts))
    priori = cal_priori()
    mus = mean_mat()


In [0]:
def bayes_c(X):
    global training_freq
    global training_data
    global test_data
    global m0,s0
    global sigs
    global dets
    global priori
    clas = ""
    max = 0
    dik={}
    for i in training_freq.keys():
        dik[i] = mulvar_nor(X,i)+math.log(priori.get(i))
    pm = -999999
    for v in dik.keys():
        if dik.get(v)>pm:
            pm = dik.get(v)
            clas = v
    #print str(te)+ " " + str(mulvar_nor(X,j)*priori.get(j))
    return clas


In [0]:
def test_rn():
    fp = 0
    cp = 0
    fn = 0
    cn = 0
    p = 0
    f = 0
    t = 0
    for v,u in test_data:
        t+=1
        k = str(bayes_c(u))
        if k==str(v):
            p+=1
        else:
            f+=1
    print "accuracy = " + str(float(p)/float(t))


In [0]:
def pca(X):
    a = []
    for i in X:
        M = np.mean(i)
        i = i-M
        b = i.reshape(28,28)
        c = np.cov(b)
        values,vectors = np.linalg.eig(c)
        eig_pair = [(np.abs(values[i]),vectors[:,i]) for i in range(len(values))]
        eig_pair.sort(key=lambda x:x[0])
        eig_pair.reverse()
        ab = []
        for j in range():
            ab = np.concatenate((ab,eig_pair[j][1]),axis = None)
        a.append(ab)
    return a


In [0]:
train_rn()
test_rn()

![alt text](https://2.bp.blogspot.com/-dvmGCHt1IsE/W57oy9mGLOI/AAAAAAAAKPE/FfqahezKlzgq0DvW_5n1JcMKfEWd4tJWQCLcBGAs/s320/Capture.PNG)

# Part B
Medical Data

In [0]:
mus = []
training_freq = {}
training_data = []
test_data = []
sigs = []
dets = 1
priori = []

In [0]:
def cal_priori():
    global training_freq
    global training_data
    global test_data
    global m0,s0
    global sigs
    global dets
    global priori
    global mus
    dic = training_freq.copy()
    for i in dic.keys():
        dic[i] = float(dic.get(i))/float(len(training_data))
    priori = dic.copy()
    return dic

In [0]:
def mean_mat():
    global training_freq
    global training_data
    global test_data
    global m0,s0
    global sigs
    global dets
    global priori
    global mus
    dic = {}
    for y,x in training_data:
        if y not in dic:
            dic[y] = []
        dic[y].append(x)
    for j in dic.keys():
        dic[j] = np.mean(dic.get(j),axis=0)
    mus = dic
    return dic


In [0]:
def mulvar_nor(X,y):
    global training_freq
    global training_data
    global test_data
    global m0,s0
    global sigs
    global dets
    global priori
    global mus
    c = math.log(math.pow(2*math.pi,-0.5*len(X)))+math.log(math.pow(np.real(dets),-0.5))
    xut = np.array([X-mus.get(y)])
    sig_i = np.linalg.inv(sigs)
    pd = np.matmul(xut , sig_i)
    xu = np.array([X-mus.get(y)]).T
    e = (-0.5)*np.linalg.det(np.matmul(pd,xu))
    return c+e


In [0]:
def train_rn():
    global training_freq
    global training_data
    global test_data
    global m0,s0
    global sigs
    global dets
    global priori
    global mus
    priori = cal_priori()
    mus = mean_mat()


In [0]:
def bayes_c(X):
    global training_freq
    global training_data
    global test_data
    global m0,s0
    global sigs
    global dets
    global priori
    global mus
    clas = ""
    max = 0
    dik={}
    for i in training_freq.keys():
        dik[i] = mulvar_nor(X,i)+math.log(priori.get(i))
    pm = -999999
    for v in dik.keys():
        if dik.get(v)>pm:
            pm = dik.get(v)
            clas = v
    #print str(te)+ " " + str(mulvar_nor(X,j)*priori.get(j))
    return clas


In [0]:
def test_rn():
    fp = 0
    cp = 0
    fn = 0
    cn = 0
    p = 0
    f = 0
    t = 0
    for v,u in test_data:
        t+=1
        k = str(bayes_c(u))
        if k==str(v):
            p+=1
        else:
            f+=1
    print "accuracy = " + str(float(p)/float(t))


In [32]:
df = pd.read_csv("drive/ml1/Medical_data.csv")
y_train = np.array(df['Health'].values.tolist())
X_train = df[['TEST1','TEST2','TEST3']].values.tolist()
df = pd.read_csv("drive/ml1/test_medical.csv")
y_test = np.array(df['Health'].values.tolist())
X_test = df[['TEST1','TEST2','TEST3']].values.tolist()
sigs = np.cov(X_train,rowvar=0)
dets = np.linalg.det(sigs)
training_data = [x for x in zip(y_train,X_train)]
test_data = [x for x in zip(y_test,X_test)]
unique, counts = np.unique(y_train, return_counts=True)
training_freq = dict(zip(unique,counts))
train_rn()
test_rn()

accuracy = 0.816666666667


# Part C
Railway Data

In [0]:
mus = []
training_freq = {}
training_data = []
test_data = []
sigs = []
dets = 1
priori = []
priori_d = {}
recal = 0.5
precisio = 0.5
fscore = 0.5
k = 1
fp = 0
cp = 0
fn = 0
cn = 0
accu = 0.92

In [0]:
def cal_priori_d():
    global dets
    global k
    global training_freq
    global training_data
    global test_data
    global priori
    global sigs
    global mus
    global priori_d
    dicd = {}
    for y,x,x1 in training_data:
        if str(x1) not in dicd.keys():
            #print x1
            dicd[str(x1)] = 0
        if y==1:
            dicd[str(x1)]+=1
    for a in dicd.keys():
        dicd[a] = dicd.get(a)/float(len(training_data))
    priori_d = dicd.copy()
    return dicd

In [0]:
def cal_priori():
    global dets
    global k
    global training_freq
    global training_data
    global test_data
    global priori
    global sigs
    global mus
    global priori_d
    dic = training_freq.copy()
    for i in dic.keys():
        dic[i] = float(dic.get(i))/float(len(training_data))
    priori = dic.copy()
    return dic


In [0]:
def mean_mat():
    global dets
    global k
    global training_freq
    global training_data
    global test_data
    global priori
    global sigs
    global mus
    global priori_d
    dic = {}
    for y,x,x1 in training_data:
        if y not in dic:
            dic[y] = []
        dic[y].append(x)
    for j in dic.keys():
        dic[j] = np.mean(dic.get(j),axis=0)
    mus = dic.copy()
    return dic


In [0]:
def mulvar_nor(X,y):
    global training_freq
    global k
    global training_data
    global test_data
    global m0,s0
    global sigs
    global dets
    global priori
    global mus
    c = math.log(math.pow(2*math.pi,-0.5*len(X)))+math.log(math.pow(np.real(dets),-0.5))
    xut = np.array([X-mus.get(y)])
    sig_i = np.linalg.inv(sigs)
    pd = np.matmul(xut , sig_i)
    xu = np.array([X-mus.get(y)]).T
    e = (-0.5)*np.linalg.det(np.matmul(pd,xu))
    return c+e

In [0]:
def train_rn():
    global dets
    global k
    global training_freq
    global training_data
    global test_data
    global priori
    global sigs
    global mus
    global priori_d
    df = pd.read_csv("drive/ml1/railwayBookingList.csv")
    y_train = np.array(df['boarded'].values.tolist())
    X_train = df[['budget', 'memberCount', 'age']].values.tolist()
    sigs = np.cov(X_train,rowvar=0)
    dets = np.linalg.det(sigs)
    X_train1 = df[['preferredClass', 'sex']].values.tolist()
    training_data = [x for x in zip(y_train[:1100],X_train[:1100],X_train1[:1100])]
    #print "length of training_data: "+str(len(training_data))
    test_data = [x for x in zip(y_train[1100:],X_train[1100:],X_train1[1100:])]
    #print "length of test_data: "+str(len(test_data))
    unique, counts = np.unique(y_train, return_counts=True)
    training_freq = dict(zip(unique,counts))
    cal_priori()
    cal_priori_d()
    mean_mat()

In [0]:
def bayes_c(X,X1):
    global dets
    global k
    global training_freq
    global training_data
    global test_data
    global priori
    global sigs
    global mus
    global priori_d
    clas = 1
    max = 0
    dik ={}
    if X1 not in priori_d:
        return 1
    for i in training_freq.keys():
        if i==1:
            post_d = priori_d.get(X1)
        else:
            post_d = 1-priori_d.get(X1)
        dik[i] = mulvar_nor(X,i)*priori.get(i)*post_d
    #print str(dik.get(0))+" "+str(k*dik.get(1))
    if dik.get(0)>k*dik.get(1):
        return 0
    return 1

In [0]:
def test_rn():
    global fscore
    global precisio
    global recal
    global k
    global fp
    global cp
    global fn
    global cn
    global accu
    accu = 0.85
    p = 0
    f = 0
    t = 0
    for v,u,c in test_data:
        t+=1
        st = str(v)
        te = str(bayes_c(u,str(c)))
        if (st=="0" and te=="0"):
            cn+=1
        if (st=="0" and te=="1"):
            fp+=1
        if (st=="1" and te=="0"):
            fn+=1
        if (st=="1" and te=="1"):
            cp+=1
        if te==st:
            p+=1
        else:
            f+=1
    recal = cp/(float(cp)+ float(fn))
    precisio = cp/(float(cp)+ float(fp))
    fscore = 2*precisio*recal/(precisio + recal)
    accu = str(float(p)/float(t))
    print "Accuracy: " + str(accu)
    print "Precision: " + str(precisio)
    print "recall: " + str(recal)
    print "F1 score: " + str(fscore)


In [0]:
def roc():
    global k
    global fp
    global cp
    global fn
    global cn
    ro = [(0,0)]
    for i in range(100):
        k = -2.5 + i/float(10)
        #train_rn()
        test_rn()
        le = len(test_data)
        ro.append((fp/float(le),cp/float(le)))
        fp = 0
        cp = 0
        fn = 0
        cn = 0
    #print ro
    plt.scatter(*zip(*ro))
    plt.show()

![alt text](https://2.bp.blogspot.com/-XzCZ_uO9UHY/W57x3Qpe9tI/AAAAAAAAKPY/HT0ULZWjyd8oSfpocvIJz6H_AufDolXfgCLcBGAs/s320/Froc.png)

In [51]:
train_rn()
test_rn()
#roc()

Accuracy: 0.828571428571
Precision: 0.828571428571
recall: 1.0
F1 score: 0.90625


#Section 3
Bayesian Classifier with Bayesian Estimation

#Part A
Fashion Mnist

In [0]:
mus = []
training_freq = {}
training_data = []
test_data = []
sigs = []
dets = 1
priori = []
m0 = 1
s0 = 1
n = 1


In [0]:
def cal_priori():
    global training_freq
    global training_data
    global test_data
    global m0,s0
    global sigs
    global dets
    global priori
    global mus
    dic = training_freq.copy()
    for i in dic.keys():
        dic[i] = float(dic.get(i))/float(len(training_data))
    priori = dic.copy()
    return dic

In [0]:
def mean_mat():
    global training_freq
    global training_data
    global test_data
    global m0,s0
    global sigs
    global dets
    global priori
    global mus
    global n
    dic = {}
    for y,x in training_data:
        if y not in dic:
            dic[y] = []
        dic[y].append(x)
    for j in dic.keys():
        dic[j] = np.mean(dic.get(j),axis=0)
    ns0 = np.array(n*s0)
    sigs = sigs.astype(float)
    ns0 = ns0.astype(float)
    for va in dic.keys():
        mn0 = np.matmul(np.matmul(n*s0,np.linalg.inv(np.array(ns0+sigs))),dic.get(va))
        mn1 = np.matmul(np.matmul(sigs,np.linalg.inv(np.array(ns0+sigs))),m0)
        mn = np.array(mn0+mn1)
        dic[va] = mn
    mus = dic
    return dic


In [0]:
def mulvar_nor(X,y):
    global training_freq
    global training_data
    global test_data
    global m0,s0
    global sigs
    global dets
    global priori
    global mus
    #print dets
    #c = math.log(math.pow(2*math.pi,-0.5*len(X)))+math.log(math.pow(np.real(dets),-0.5))
    #math.log(2*math.pi) = 1.8378770664093453
    c = -0.5*len(X)*1.8378770664093453+ -0.5*abs(dets)
    xut = np.array([X-mus.get(y)])
    sig_i = np.linalg.inv(sigs)
    pd = np.matmul(xut , sig_i)
    xu = np.array([X-mus.get(y)]).T
    e = (-0.5)*np.linalg.det(np.matmul(pd,xu))
    return c+e


In [0]:
def mul_exp(X,y):
    return lam.get(y)*math.exp(-1*np.matmul(lam.get(y),X))


In [0]:
def train_rn():
    global training_freq
    global training_data
    global test_data
    global m0,s0
    global sigs
    global dets
    global priori
    global mus
    X_train, y_train = load_mnist('drive/ml1/data/fashion', kind='train')
    #X_train = np.array(pca(X_train))
    #X_train = np.array(X_train).real.astype(float)
    sigs = np.cov(X_train,rowvar=0)
    dets = np.linalg.slogdet(sigs)[-1]
    X_test, y_test = load_mnist('drive/ml1/data/fashion', kind='t10k')
    #X_test = pca(X_test)
    #X_test = np.array(X_test).real.astype(float)
    training_data = [x for x in zip(y_train,X_train[:-len(X_train)/10])]
    val_data = [x for x in zip(y_train,X_train[-len(X_train)/10:])]
    m0 = np.mean(X_train[-len(X_train)/10:],axis=0)
    s0 = np.cov(X_train[-len(X_train)/10:],rowvar=0)
    n=len(X_train)-len(X_train)/10
    sn = np.matmul(np.matmul(sigs,np.linalg.inv(np.array(n*s0+sigs))),s0)
    sigs = np.array(sn+sigs)
    test_data = [x for x in zip(y_test,X_test)]
    unique, counts = np.unique(y_train, return_counts=True)
    training_freq = dict(zip(unique,counts))
    priori = cal_priori()
    mus = mean_mat()


In [0]:
def bayes_c(X):
    global training_freq
    global training_data
    global test_data
    global m0,s0
    global sigs
    global dets
    global priori
    clas = ""
    max = 0
    dik={}
    for i in training_freq.keys():
        dik[i] = mulvar_nor(X,i)+math.log(priori.get(i))
    pm = -999999
    for v in dik.keys():
        if dik.get(v)>pm:
            pm = dik.get(v)
            clas = v
    #print str(te)+ " " + str(mulvar_nor(X,j)*priori.get(j))
    return clas


In [0]:
def test_rn():
    fp = 0
    cp = 0
    fn = 0
    cn = 0
    p = 0
    f = 0
    t = 0
    for v,u in test_data:
        t+=1
        k = str(bayes_c(u))
        if k==str(v):
            p+=1
        else:
            f+=1
    print "accuracy = " + str(float(p)/float(t))


In [0]:
def pca(X):
    a = []
    for i in X:
        M = np.mean(i)
        i = i-M
        b = i.reshape(28,28)
        c = np.cov(b)
        values,vectors = np.linalg.eig(c)
        eig_pair = [(np.abs(values[i]),vectors[:,i]) for i in range(len(values))]
        eig_pair.sort(key=lambda x:x[0])
        eig_pair.reverse()
        ab = []
        for j in range():
            ab = np.concatenate((ab,eig_pair[j][1]),axis = None)
        a.append(ab)
    return a

In [0]:
train_rn()
test_rn()

![alt text](https://2.bp.blogspot.com/-aDBDQgbZ5k8/W57o1JoQ9yI/AAAAAAAAKPI/8rW8eReCTDca9AnH--smS7d8QCm1fu_DQCLcBGAs/s320/Capture2.PNG)

#Part B
Medical Data

In [0]:
mus = []
training_freq = {}
training_data = []
test_data = []
sigs = []
dets = 1
priori = []
m0 = 1
s0 = 1
n = 1

In [0]:
def cal_priori():
    global training_freq
    global training_data
    global test_data
    global m0,s0
    global sigs
    global dets
    global priori
    global mus
    dic = training_freq.copy()
    for i in dic.keys():
        dic[i] = float(dic.get(i))/float(len(training_data))
    priori = dic.copy()
    return dic


In [0]:
def mean_mat():
    global training_freq
    global training_data
    global test_data
    global m0,s0
    global sigs
    global dets
    global priori
    global mus
    global n
    dic = {}
    for y,x in training_data:
        if y not in dic:
            dic[y] = []
        dic[y].append(x)
    for j in dic.keys():
        dic[j] = np.mean(dic.get(j),axis=0)
    ns0 = np.array(n*s0)
    sigs = sigs.astype(float)
    ns0 = ns0.astype(float)
    for va in dic.keys():
        mn0 = np.matmul(np.matmul(n*s0,np.linalg.inv(np.array(ns0+sigs))),dic.get(va))
        mn1 = np.matmul(np.matmul(sigs,np.linalg.inv(np.array(ns0+sigs))),m0)
        mn = np.array(mn0+mn1)
        dic[va] = mn
    mus = dic
    return dic


In [0]:
def mulvar_nor(X,y):
    global training_freq
    global training_data
    global test_data
    global m0,s0
    global sigs
    global dets
    global priori
    global mus
    #print dets
    #c = math.log(math.pow(2*math.pi,-0.5*len(X)))+math.log(math.pow(np.real(dets),-0.5))
    #math.log(2*math.pi) = 1.8378770664093453
    c = -0.5*len(X)*1.8378770664093453+ -0.5*abs(dets)
    xut = np.array([X-mus.get(y)])
    sig_i = np.linalg.inv(sigs)
    pd = np.matmul(xut , sig_i)
    xu = np.array([X-mus.get(y)]).T
    e = (-0.5)*np.linalg.det(np.matmul(pd,xu))
    return c+e


In [0]:
def mul_exp(X,y):
    return lam.get(y)*math.exp(-1*np.matmul(lam.get(y),X))


In [0]:
def train_rn():
    global training_freq
    global training_data
    global test_data
    global m0,s0
    global sigs
    global dets
    global priori
    global mus
    priori = cal_priori()
    mus = mean_mat()


In [0]:
def bayes_c(X):
    global training_freq
    global training_data
    global test_data
    global m0,s0
    global sigs
    global dets
    global priori
    clas = ""
    max = 0
    dik={}
    for i in training_freq.keys():
        dik[i] = mulvar_nor(X,i)+math.log(priori.get(i))
    pm = -999999
    for v in dik.keys():
        if dik.get(v)>pm:
            pm = dik.get(v)
            clas = v
    #print str(te)+ " " + str(mulvar_nor(X,j)*priori.get(j))
    return clas


In [0]:
def test_rn():
    fp = 0
    cp = 0
    fn = 0
    cn = 0
    p = 0
    f = 0
    t = 0
    for v,u in test_data:
        t+=1
        k = str(bayes_c(u))
        if k==str(v):
            p+=1
        else:
            f+=1
    print "accuracy = " + str(float(p)/float(t))


In [0]:
def pca(X):
    a = []
    for i in X:
        M = np.mean(i)
        i = i-M
        b = i.reshape(28,28)
        c = np.cov(b)
        values,vectors = np.linalg.eig(c)
        eig_pair = [(np.abs(values[i]),vectors[:,i]) for i in range(len(values))]
        eig_pair.sort(key=lambda x:x[0])
        eig_pair.reverse()
        ab = []
        for j in range():
            ab = np.concatenate((ab,eig_pair[j][1]),axis = None)
        a.append(ab)
    return a


In [75]:
df = pd.read_csv("drive/ml1/Medical_data.csv")
y_train = np.array(df['Health'].values.tolist())
X_train = df[['TEST1','TEST2','TEST3']].values.tolist()
df = pd.read_csv("drive/ml1/test_medical.csv")
y_test = np.array(df['Health'].values.tolist())
X_test = df[['TEST1','TEST2','TEST3']].values.tolist()
sigs = np.cov(X_train,rowvar=0)
dets = np.linalg.slogdet(sigs)[-1]
test_data = [x for x in zip(y_test,X_test)]
training_data = [x for x in zip(y_train,X_train[:-len(X_train)/10])]
val_data = [x for x in zip(y_train,X_train[-len(X_train)/10:])]
m0 = np.mean(X_train[-len(X_train)/10:],axis=0)
s0 = np.cov(X_train[-len(X_train)/10:],rowvar=0)
n=len(X_train)-len(X_train)/10
sn = np.matmul(np.matmul(sigs,np.linalg.inv(np.array(n*s0+sigs))),s0)
sigs = np.array(sn+sigs)
unique, counts = np.unique(y_train, return_counts=True)
training_freq = dict(zip(unique,counts))
train_rn()
test_rn()

accuracy = 0.817


# Section 3
Naive Bayes Classifier

# Part A
Medical Data

In [0]:
def loadCsv(filename):
	lines = csv.reader(open(filename, "rb"))
	dataset = list(lines)
	leng = len(dataset)
	dataset = dataset[1:leng]
	return dataset
def mean(arr):
	arr = arr.astype(np.float)
	return np.sum(arr)/arr.size

def standev(arr):
	arr = arr.astype(np.float)
	mea = mean(arr);
	squaresum = sum(np.power(arr-mea,2))/float(arr.size)
	return math.sqrt(squaresum) 
def trainprob(arr):
	arr = np.array(arr)
	unique, counts = np.unique(arr, return_counts=True)
	counts = np.float_(counts)/float(arr.size)
	return (unique,counts)
def probdata(x,mean,stdev):
	exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
	return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent
def dsepbyclass(dataset):
	dicte = {}
	for i in dataset:
		if(i[0] not in dicte):
			dicte[i[0]] = []
		dicte[i[0]].append(i[1:4])
	return dicte
def meanvar(dataset):
	dicte = {}
	for i in dataset:
		a = zip(*dataset[i])
		dicte[i] = []
		for j in a:
			p = np.array(j)
			dicte[i].append([mean(p),standev(p)])
	return dicte

In [86]:
def predictclass(mdict,data,probmatrix):
	dicter = predictprob(mdict,data)
	bestprob,bestclass = -1,None
	t =0
	for clas,prob in dicter.iteritems():
		if bestprob<prob*probmatrix[t] :
			bestprob = prob*probmatrix[t]
			bestclass = clas
		t+=1

	return bestclass


   
def predictprob(mdict,data):
	dicter = {}
	
	for classvalue,classdata in mdict.iteritems():
		prob = 1
		t=1
		for j in classdata:
			prob *= probdata(float(data[t]),j[0],j[1])
			t +=1
		dicter[classvalue] = prob
	return dicter


            




	#return [(mean(x),standev(x)) for x in a]
def answerdata(file1,file2):
	dataset = loadCsv(file1)
	dic = dsepbyclass(dataset)
	mdict = meanvar(dic)
	dataset1 = loadCsv(file2)
	dataset1 = dataset1[400:700]
	unique ,counts = trainprob(dataset[1:500])
	accuracy =0
	for j in dataset1:
		a =  predictclass(mdict,j,counts)
		if(a==j[0]):
			accuracy += 1
	print (float(accuracy)/float(len(dataset1)))*100



answerdata("drive/ml1/Medical_data.csv","drive/ml1/test_medical.csv")

89.6666666667


# Part B
Fashion Mnist Data

In [0]:
def loadCsv(filename):
	lines = csv.reader(open(filename, "rb"))
	dataset = list(lines)
	leng = len(dataset)
	dataset = dataset[1:leng]
	return dataset
def mean(arr):
	arr = arr.astype(np.float)

	return np.sum(arr)/arr.size

def standev(arr):
	arr = arr.astype(np.float)
	mea = mean(arr)
	squaresum = sum(np.power(arr-mea,2))/float(arr.size)
	return math.sqrt(squaresum)
def trainprob(dicte,X_train):
	#unique, counts = np.unique(arr, return_counts=True)
	#counts = np.float_(counts)/float(arr.size)
	#return (unique,counts)
	a ={}
	lent = len(X_train)
	for j in dicte:
		a[j] =   float(len(dicte[j]))/float(lent)
	return a
def probdata(x,mean,stdev):
	#print mean
	#print stdev
	if stdev==0:
		return 1
	exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
	return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent
def dsepbyclass(X_train,y_train):
	dicte = {}
	for i in range(len(X_train)):
		if(y_train[i] not in dicte):
			dicte[y_train[i]] = []
		dicte[y_train[i]].append(X_train[i])
	return dicte
def meanvar(dataset):
	dicte = {}
	for i in dataset:
		a = zip(*dataset[i])
		#print len(a)
		dicte[i] = []
		for j in a:
			p = np.array(j)
			dicte[i].append([mean(p),standev(p)])
	return dicte
def predictclass(mdict,data,probmatrix):
	dicter = predictprob(mdict,data)
	bestprob,bestclass = -1,None
	t =0
	for clas,prob in dicter.iteritems():
		if bestprob<prob*probmatrix[clas] :
			bestprob = prob*probmatrix[clas]
			bestclass = clas
		t+=1

	return bestclass



def predictprob(mdict,data):
	dicter = {}

	for classvalue,classdata in mdict.iteritems():
		prob = 1
		t=0
		for j in classdata:
			#print j
			prob *= probdata(float(data[t]),j[0],j[1])
			t +=1
		dicter[classvalue] = prob
	return dicter

	#return [(mean(x),standev(x)) for x in a]
def answerdata():
	X_train, y_train = load_mnist("drive/ml1/data/fashion", kind='train')
	X_test, y_test = load_mnist("drive/ml1/data/fashion", kind='t10k')
	#pcareduction([X_train[0]])
	#return
	X_test = pcareduction(X_test)
	X_train = pcareduction(X_train)
	#dataset = loadCsv(file1)
	dic = dsepbyclass(X_train,y_train)
	mdict = meanvar(dic)
	counts = trainprob(dic,X_train)
	accuracy =0
	for j in range(len(X_test)):
		a =  predictclass(mdict,X_test[j],counts)
		if(a==y_test[j]):
			accuracy += 1
			#print "correct"
		#else:
			#print "incorrect"
	print "Accuracy: " + str((float(accuracy)/float(len(X_test)))*100)
def pcareduction(X_train):
	a = []
	for i in X_train:
	    M = np.mean(i)
	    i = i-M
	    b = i.reshape(28,28)
	    #m = np.mean(b.T,axis =1)
	    c = np.cov(b)
	    values,vectors = np.linalg.eig(c)
	    eig_pair = [(np.abs(values[i]),vectors[:,i]) for i in range(len(values))]
	    eig_pair.sort(key=lambda x:x[0])
	    eig_pair.reverse()
	    #print eig_pair
	    ab = []
	    #print eig_pair[0][0]
	    for j in range(6):
	    	ab = np.concatenate((ab,eig_pair[j][1]),axis = None)
	    #print ab
	    a.append(ab)
	return a




def pca1(X_train):
	a = []
	b = X_train.T
	M = np.mean(b,axis=1)
	print M.shape
	ac = X_train- M
	c = np.cov(ac.T)
	print c.shape
	values,vectors = np.linalg.eig(c)
	eig_pair = [(np.abs(values[i]),vectors[:,i]) for i in range(len(values))]
	eig_pair.sort(key=lambda x:x[0])
	eig_pair.reverse()
	for i,j in eig_pair:
		print i
	ab =[]
	for j in range(6):
		ab.append(eig_pair[j][1])
	ab = np.array(ab)
	print ab.shape
	return
	p = ab.dot(ac.T)
	print p.shape
	return p.T
def a(x):
	x.sort(key=lambda x:x[0])
	

In [95]:
answerdata()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:13: ComplexWarning: Casting complex values to real discards the imaginary part
  del sys.path[0]
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:72: ComplexWarning: Casting complex values to real discards the imaginary part


Accuracy: 52.48


#Part C
Railway Data

In [96]:
def loadCsv(filename):
	lines = csv.reader(open(filename, "rb"))
	dataset = list(lines)
	leng = len(dataset)
	dataset = np.array(dataset[1:leng])
	dataset1 = dataset[:,[1,2,3,6]]
	dataset2 = dataset[:,[1,4,5]]
	return dataset1,dataset2
def mean(arr):
	arr = arr.astype(np.float)
	return np.sum(arr)/arr.size

def standev(arr):
	arr = arr.astype(np.float)
	mea = mean(arr);
	squaresum = sum(np.power(arr-mea,2))/float(arr.size)
	return math.sqrt(squaresum) 
def trainprob(arr):
	arr = np.array(arr)
	unique, counts = np.unique(arr, return_counts=True)
	counts = np.float_(counts)/float(arr.size)
	return (unique,counts)
def probdata(x,mean,stdev):
	exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
	return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent
def dsepbyclass(dataset):
	dicte = {}
	for i in dataset:
		if(i[0] not in dicte):
			dicte[i[0]] = []
		dicte[i[0]].append(i[1:4])
	return dicte
def meanvar(dataset):
	dicte = {}
	for i in dataset:
		a = zip(*dataset[i])
		dicte[i] = []
		for j in a:
			p = np.array(j)
			dicte[i].append([mean(p),standev(p)])
	return dicte
def newprob(dicter):
	dic ={}
	for j in dicter:
		a = zip(*dicter[j])
		unique1,counts1 = np.unique(a[0], return_counts=True)
		unique2,counts2 = np.unique(a[1], return_counts=True) 
		b = {}
		c =0
		d= 0 
		for k in counts1:
			c +=k
		for l in counts2:
			d +=l
		for qw in range(len(unique1)):
			b[unique1[qw]] = float(counts1[qw])/float(c)
		for qa in range(len(unique2)):
			b[unique2[qa]] = float(counts2[qa])/float(c)
		dic[j] = b
	if len(dic['0']) != len(dic['1']):
		for j in dic['0']:
			if not j in dic['1']:
				dic['1'][j] =0
		for j in dic['1']:
			if not j in dic['0']:
				dic['0'][j] =0




    
	return dic
def predictclass(mdict,data,data1,probmatrix,ndict):
	dicter = predictprob(mdict,data,probmatrix)
	bestprob,bestclass = -1,None
	#print dicter
	for clas,prob in dicter.iteritems():
		
		ab = ndict[clas]
		if data1[1] not in ab:
			p=0
		elif data1[2] not in ab:
			p = 0
		else:
			p = prob*ab[data1[1]]*ab[data1[2]]
		
		if bestprob< p:
			bestprob = p
			bestclass = clas

	return bestclass


   
def predictprob(mdict,data,cd):
	dicter = {}
	re = 0
	for classvalue,classdata in mdict.iteritems():
		prob = 1
		t=1
		for j in classdata:
			prob *= probdata(float(data[t]),j[0],j[1])
			t +=1
		dicter[classvalue] = prob*cd[re]
		re += 1
	return dicter

	#return [(mean(x),standev(x)) for x in a]
def answerdata(file1,file2):
	dataset1,dataset2 = loadCsv(file1)
	dic = dsepbyclass(dataset1)
	dic1 = dsepbyclass(dataset2)
	mdict = meanvar(dic)
	ndict = newprob(dic1)
	dataset3 ,dataset4 = loadCsv(file2)
	#print trainprob
	unique ,counts = trainprob(dataset1[:,1])
	accuracy =0
	tp = 0
	fp =0
	fn =0
	for s in range(len(dataset3)):
		j = dataset3[s]
		k = dataset4[s]
		a =  predictclass(mdict,j,k,counts,ndict)
		if(a==j[0]):
			accuracy += 1
			if(a =="1"):
				tp +=1
		else:
			if(a=="1"):
				fp +=1
			if(a=="0"):
				fn += 1
	precision = float(tp)/float(tp+fp)
	recall = float(tp)/float(tp+fn)
	print "accuaracy = " + str((float(accuracy)/float(len(dataset1)))*100)
	print "precision = " + str(precision)
	print "recall = " + str(recall)
	print "F1 score = " +str(float(2*precision*recall)/float(precision+recall))



answerdata("drive/ml1/railwayBookingList.csv","drive/ml1/railwayBookingList.csv")

accuaracy = 76.0305343511
precision = 0.745059288538
recall = 0.930864197531
F1 score = 0.827661909989


#Section 4
K Nearest Neighbor

#Part A
Medical Data

In [99]:
def knnclassifier(file1,file2,k):
	dataset1 = np.array(loadCsv(file1))
	dataset1 = dataset1[1:1500]
	dataset2 = np.array(loadCsv(file2))
	dataset2 = dataset2[1500:3000]

	unique,counts  = np.unique(dataset1[:,0],return_counts=True)
	accuracy =0
	for i in dataset2:
		#print i
		clas = find(i,dataset1,k)
		#print "---"
		#print clas
		#print"-----"
		if clas == i[0]:
			accuracy +=1
	print (float(accuracy)/float(len(dataset2)))*100




def find(data,file,k):
	a = []
	for j in file:
		dis = finddis(data,j)
		a.append([j[0],dis])
	#print len(a)
	b = findclass(a,k)
	return b

def findclass(a,k):
	m = {}
	b =[]
	n = len(a)
	for i in range(k):
		p =float("inf")
		c = ''
		for j in range(i,n-1):
			if a[j][1] <p:
				p = a[j][1]
				c = a[j][0] 
		b.append([c,p])
	#print b
	for d in range(k-1):
		if b[d][0] not in m:
			m[b[d][0]] =1
		else:
			m[b[d][0]] +=1
	maxclass = None
	maxvalue = -1
	for clas,value in m.iteritems():
		if maxvalue<value:
			maxvalue = value
			maxclass = clas
	return maxclass

def finddis(data1,data2):
	a = np.array(data1[1:3])
	a = a.astype(np.float)
	b = np.array(data2[1:3])
	b = b.astype(np.float)
	return math.sqrt(np.sum((a-b)**2))

def loadCsv(filename):
	lines = csv.reader(open(filename, "rb"))
	dataset = list(lines)
	leng = len(dataset)
	dataset = dataset[1:leng]
	return dataset
knnclassifier("drive/ml1/Medical_data.csv","drive/ml1/test_medical.csv",50)

79.6


#Part B
Railway Data

In [100]:
def knnclassifier(file1,file2,k):
	dataset1 = np.array(loadCsv(file1))
	dataset2 = np.array(loadCsv(file2))
	dataset11 = dataset1[:,[1,2,3,6]]
	dataset12 = dataset1[:,[1,4,5]]
	dataset21 = dataset2[:,[1,2,3,6]]
	dataset22 = dataset2[:,[1,4,5]]

	unique,counts  = np.unique(dataset1[:,0],return_counts=True)
	accuracy =0
	fp =0
	fn=0
	tp = 0
	for i in range(len(dataset2)):
		#print i
		clas = find(dataset21[i],dataset22[i],dataset11,dataset22,k)
		#print "---"
		#print clas
		#print"-----"
		if clas == dataset21[i][0]:
			accuracy +=1
			if(clas =="1"):
				tp +=1
		else:
			if(clas=="1"):
				fp +=1
			if(clas=="0"):
				fn += 1

	accuracy =  (float(accuracy)/float(len(dataset2)))*100
	precision = float(tp)/float(tp+fp)
	recall = float(tp)/float(tp+fn)
	print "accuaracy = " + str(accuracy)
	print "precision = " + str(precision)
	print "recall = " + str(recall)
	print "F1 score = " +str(float(2*precision*recall)/float(precision+recall))





def find(data,data1,file,file1,k):
	a = []
	for j in range(len(file)):
		dis = finddis(data,data1,file[j],file1[j])
		a.append([file[j][0],dis])
	#print len(a)
	b = findclass(a,k)
	return b

def findclass(a,k):
	m = {}
	b =[]
	n = len(a)
	for i in range(k):
		p =float("inf")
		c = ''
		for j in range(i,n-1):
			if a[j][1] <p:
				p = a[j][1]
				c = a[j][0] 
		b.append([c,p])
	#print b
	for d in range(k-1):
		if b[d][0] not in m:
			m[b[d][0]] =1
		else:
			m[b[d][0]] +=1
	maxclass = None
	maxvalue = -1
	for clas,value in m.iteritems():
		if maxvalue<value:
			maxvalue = value
			maxclass = clas
	return maxclass

def finddis(data1,data3,data2,data4):
	a = np.array(data1[1:3])
	a = a.astype(np.float)
	b = np.array(data2[1:3])
	b = b.astype(np.float)
	c = math.sqrt(np.sum((a-b)**2))
	if data3[1] != data4[1]:
		c+=2
	if data3[2] != data4[2]:
		c+=2
	return c

def loadCsv(filename):
	lines = csv.reader(open(filename, "rb"))
	dataset = list(lines)
	leng = len(dataset)
	dataset = dataset[1:leng]
	return dataset
knnclassifier("drive/ml1/railwayBookingList.csv","drive/ml1/railwayBookingList.csv",25)

accuaracy = 79.7709923664
precision = 0.827130852341
recall = 0.850617283951
F1 score = 0.838709677419


#Part C
Fashion Mnist

In [0]:
def pcareduction(X_train):
	a = []
	for i in X_train:
	    M = np.mean(i)
	    i = i-M
	    b = i.reshape(28,28)
	    c = np.cov(b)
	    values,vectors = np.linalg.eig(c)
	    eig_pair = [(np.abs(values[i]),vectors[:,i]) for i in range(len(values))]
	    eig_pair.sort(key=lambda x:x[0])
	    eig_pair.reverse()
	    #print eig_pair
	    ab = []
	    #print eig_pair[0][0]
	    for j in range(6):
	    	ab = np.concatenate((ab,eig_pair[j][1]),axis = None)
	    #print ab
	    a.append(ab)
	#print a



	return a

def knnclassifier(k):
	X_train, y_train = mnist_reader.load_mnist("drive/ml1/data/fashion", kind='train')
	X_test, y_test = mnist_reader.load_mnist("drive/ml1/data/fashion", kind='t10k')
	X_train  = pcareduction(X_train)
	X_test = pcareduction(X_test)
	y_train = np.reshape(y_train, (-1, 1))
	y_test = np.reshape(y_test, (-1, 1))
	dataset1 = np.hstack((y_train,X_train))
	dataset2 = np.hstack((y_test,X_test))
	dataset1 = dataset1
	dataset2 = dataset2
	unique,counts  = np.unique(dataset1[:,0],return_counts=True)
	accuracy =0
	for i in dataset2:
		#print i
		clas = find(i,dataset1,k)
		#print "---"
		#print clas
		#print"-----"
		if clas == i[0]:
			accuracy +=1
			#print "correct"
		#else :
			#print "incorrect"
	print "accurracy=" + str((float(accuracy)/float(len(dataset2)))*100)




def find(data,file,k):
	a = []
	for j in file:
		dis = finddis(data,j)
		a.append([j[0],dis])
	#print len(a)
	b = findclass(a,k)
	return b

def findclass(a,k):
	m = {}
	b =[]
	n = len(a)
	for i in range(k):
		p =float("inf")
		c = ''
		for j in range(i,n-1):
			if a[j][1] <p:
				p = a[j][1]
				c = a[j][0]
		b.append([c,p])
	#print b
	for d in range(k-1):
		if b[d][0] not in m:
			m[b[d][0]] =1
		else:
			m[b[d][0]] +=1
	maxclass = None
	maxvalue = -1
	for clas,value in m.iteritems():
		if maxvalue<value:
			maxvalue = value
			maxclass = clas
	return maxclass

def finddis(data1,data2):
	a = np.array(data1[1:len(data1)])
	a = a.astype(np.float)
	b = np.array(data2[1:len(data2)])
	b = b.astype(np.float)
	return math.sqrt(np.sum((a-b)**2))

def loadCsv(filename):
	lines = csv.reader(open(filename, "rb"))
	dataset = list(lines)
	leng = len(dataset)
	dataset = dataset[1:leng]
	return dataset



In [0]:
knnclassifier(100)

![alt text](https://3.bp.blogspot.com/-VaR_5oYII5U/W57oNE53MPI/AAAAAAAAKO0/s1FnDpGl07g2yDGNl0ve6lWdXQCliP_3gCLcBGAs/s320/Screenshot_5.png)

#Section 5
K Means Clustering

In [0]:

def precision(predicted_test,test):
    TP = 0
    FP = 0
    for i in range(len(test)):
        if predicted_test[i][1] == test[i][1]:
            if predicted_test[i][1]==1:
                TP = TP + 1
        else:
            if predicted_test[i][1] == 1:
                FP = FP + 1
    #print (str(TP)+" "+str(FP))
    return TP*(1.0/(TP+FP))
def accuracy(predicted_test,test):
    T = 0
    for i in range(len(test)):
        if predicted_test[i][1] == test[i][1]:
            T = T + 1
    #print (str(T)+" ")
    return T*(1.0/len(test))
def recall(predicted_test,test):
    TP = 0
    FN = 0
    for i in range(len(test)):
        if predicted_test[i][1] == test[i][1]:
            if predicted_test[i][1]==1:
                TP = TP + 1
        else:
            if predicted_test[i][1] == 0:
                FN = FN + 1

    return TP*(1.0/(TP+FN))
def F1(precision,recall):
    return 2.0*(precision*recall)*(1.0/(precision+recall))




class kmean:
    name = ""
    k=2
    def __init__(self,name="K-Means Clustering",k=2):
        self.name = name
        self.k = k

    def initialize(self,train):

        l=[]
        i=0
        for t in train:
            if t[1]==i:
                l.append(t[0])
                i+=1
            if i==self.k:
                break
        print("number of classes: "+str(len(l)))
        return l

    def make_cluster(self,train,centroids):
        clusters = {c: [] for c in range(self.k)}
        #initialze cluster with key as label

        for X,Y in train:
            min = float('inf')
            its_cluster = 0

            for i in range(self.k):
                centroid_i = centroids[i]
                euclidean_dis = np.linalg.norm(X-centroid_i)
                if euclidean_dis<min:
                    its_cluster = i
                    min = euclidean_dis
            clusters[its_cluster].append((Y,X))
        return clusters.values()

    def avg(self,cluster):
        #print(len(cluster))
        l = list(map(lambda x: x[1], cluster))
        return np.array(l).mean(axis=0)
    def change_centroids(self,clusters):
        centroids = []
        #print(len(clusters))
        for cluster in clusters:
            #print(len(cluster))
            centroids.append(self.avg(cluster))

        return centroids

    def train_the_model(self,train):
        centroids = self.initialize(train)
        clusters = self.make_cluster(train,centroids)
        diff = 0
        i=0
        while True:
            old_centroids = centroids
            centroids = self.change_centroids(clusters)
            clusters = self.make_cluster(train,centroids)
            #print("diff",end="\r")
            #print(list(map(lambda a,b : np.linalg.norm(a-b),old_centroids,centroids)))
            new_diff = max(list(map(lambda a,b : np.linalg.norm(a-b),old_centroids,centroids)))
            #print ("new_diff-: "+ str(new_diff),end="\r")
            try:
                change_in_diff = abs((new_diff-diff)/np.mean([diff,new_diff])) * 100
                #print ("iteration-:"+str(i) + " " +str(change_in_diff) ,end="\r")
            except:
                break
            if np.isnan(change_in_diff):
                break

            diff = new_diff
            i += 1
        return clusters,centroids

    def labelling(self,clusters,centroids):
        new_centroids = []
        i=0
        for cluster in clusters:
            labels = [y for y,x in cluster]
            high_freq = max(set(labels),key=labels.count)
            centroid = (high_freq,centroids[i])
            new_centroids.append(centroid)
            i = i + 1
        return new_centroids

    def test(self,testimage,centroids):
        min = float("inf")
        y = 0
        for (label, centroid) in centroids:
            distance = np.linalg.norm(centroid - testimage)
            if distance < min:
                min = distance
                y = label
        #print(y)
        return y
def main(train,test,k):
    #print("Creating classifier Object...")
    
    classifier = kmean(k=k)
    #print("classifier name: " + classifier.name)
    #print("Training Start Here...")
    
    clusters,centroids=classifier.train_the_model(train)
    labelled_centroids = classifier.labelling(clusters,centroids)
    predicted_test = []
    #print("Prediction Start Here...")
    
    for x,y in test:
        yy = classifier.test(x,labelled_centroids)
        predicted_test.append((x,yy))

    predicted_train = []
    for x,y in train:
        yy = classifier.test(x,labelled_centroids)
        predicted_train.append((x,yy))

    print("Test Accuracy: " + str(accuracy(predicted_test,test)))
    #print("Training Accuracy: " + str(accuracy(predicted_train,test)))

    if k==2:
        p=precision(predicted_test,test)
        r=recall(predicted_test,test)
        print("Precision "+str(p))
        print("Recall "+str(r))
        print("F1 Score "+ str(F1(p,r)))



def mnist():
    trainX , trainY = load_mnist('drive/ml1/data/fashion', kind='train')
    testX , testY = load_mnist('drive/ml1/data/fashion', kind='t10k')
    train = [list(a) for a in zip(trainX, trainY)]
    test = [list(a) for a in zip(testX, testY)]

    main(train,test,10)

def medical():
    df = pd.read_csv('drive/ml1/Medical_data.csv')
    df2 = pd.read_csv('drive/ml1/test_medical.csv')
    df['Health'] = df['Health'].map({'HEALTHY':0,'MEDICATION':1,'SURGERY':2})
    df2['Health'] = df2['Health'].map({'HEALTHY':0,'MEDICATION':1,'SURGERY':2})
    trainY = df['Health'].values.tolist()
    trainXX = df[['TEST1','TEST2','TEST3']].values.tolist()
    trainX = list(map(lambda x: np.array(x),trainXX))
    train = [list(a) for a in zip(trainX, trainY)]
    testY = df2['Health'].values.tolist()
    testXX = df2[['TEST1','TEST2','TEST3']].values.tolist()
    testX = list(map(lambda x: np.array(x),testXX))
    test = [list(a) for a in zip(testX, testY)]
    #print(type(train))
    main(train,test,3)

def rail():
    df = pd.read_csv('drive/ml1/railwayBookingList.csv')
    dataY = df['boarded'].values.tolist()
    df.memberCount = df.memberCount*df['budget'].mean()
    df.age = df.age*df['budget'].mean()
    df['preferredClass'] = df['preferredClass'].map({'FIRST_AC':100000,'SECOND_AC':500000,'THIRD_AC':100000,'NO_PREF':0})
    df['sex'] = df['sex'].map({'female':300000,'male':300000})
    dataXX = df[['budget','memberCount','preferredClass','sex','age']].values.tolist()
    dataX = list(map(lambda x: np.array(x),dataXX))
    data = [list(a) for a in zip(dataX, dataY)]
    train = data[:int(len(data)*0.75)]
    test = data[int(len(data)*0.75):]
    #print(df[['budget','memberCount','preferredClass','sex','age']])

    main(train,test,2)




In [115]:
medical()


number of classes: 3
Test Accuracy: 0.534666666667


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:103: RuntimeWarning: invalid value encountered in double_scalars


In [116]:
rail()

number of classes: 2
Test Accuracy: 0.743902439024
Precision 0.746177370031
Recall 0.995918367347
F1 Score 0.853146853147


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:103: RuntimeWarning: invalid value encountered in double_scalars


In [117]:
mnist()

number of classes: 10


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:103: RuntimeWarning: invalid value encountered in double_scalars


Test Accuracy: 0.5275
